In [328]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else cpu)
print(f"Using {device} device")

torch.set_default_tensor_type('torch.cuda.FloatTensor')

Using cuda device


In [329]:
# Further setting up

# Set the seed for replicable results
seed = 0
np.random.seed(seed)

# Helper variable
lb = 1e-3 # Define lower bound for c, h
eps = 1e-5 # Define inverse punishment value for negative predictions

### Model calibration

In [330]:
# DEFINE SCALAR PARAMETERS
alpha = 0.1
beta = 0.99**25
gamma = 3
J = 3
jret = 2
thetaret = 0.2
delta = (1 + 0.015)**25 - 1
Gmu = (1 + 0.005)**25
Gchi = (1 + 0.02)**25
Pic = (1 + 0.02)**25
Hs = 0.79/5

# COMPUTE PARAMETER TRANSFORMATIONS
betahat = Gchi**((1-alpha)*(1-gamma))*beta
Pih = Gchi * Pic

# DEFINE AGE-VARYING PRODUCTIVITY AND SURVIVAL RATES
chi1 = 0.874264
mu1 = 0.493497

chi = chi1 * np.array([1, 1.5])

zeta = np.array([0.976163, 0.784336, 0])
zeta_0 = np.concatenate(([1], zeta[0:2]))
zeta_0 = torch.from_numpy(zeta_0).to(device)

# COMPUTE POPULATION DISTRIBUTION
mu = np.zeros(J)
mu[0] = mu1
for j in range(1, J):
    mu[j] = (zeta[j-1] / Gmu) * mu[j-1]

# COMPUTE TAX RATE
tau = np.sum([thetaret*Gchi**(jret - j)*mu[j] for j in range(jret, J)]) / np.sum(mu[0:jret]*chi)
mu = torch.from_numpy(mu).to(device)

# COMPUTE AGE-VARYING INCOME
y = torch.zeros(J)
for j in range(J):
    if j < jret:
        y[j] = (1-tau) * chi[j]
    else:
        y[j] = thetaret * Gchi ** (jret-j)



In [331]:
# Neural network architecture parameters
input_size = 2 + 3*J  # Dimension of extended state space (4 aggregate quantities and 3 distributions)
output_size = J + J-1 + 2  # Output dimension: 3 for housing, 2 for assets, 2 aggregate quantities

# 2. Neural network

In [332]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.SiLU(),
            nn.Linear(128, 64),
            nn.SiLU(),
            nn.Linear(64, output_size)
        )
       
    def forward(self, x):
        return self.linear_relu_stack(x)

model = NeuralNetwork(input_size, output_size).to(device)
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=11, out_features=128, bias=True)
    (1): SiLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): SiLU()
    (4): Linear(in_features=64, out_features=7, bias=True)
  )
)


In [333]:
X = torch.rand(size=(1, input_size))
# X.to(device)

In [334]:
def unpack_output(output):
    """
    Unpacks the output of the neural network

    Parameters
    ----------
    output : tensor
        Output of neural network
    """

    # Unpack simulated housing and financial assets
    R_orig = torch.unsqueeze(output[:, 0], 1)
    ph_orig = torch.unsqueeze(output[:, 1], 1)
    a_out = output[:, 2:4]
    h_out_orig = output[:, 4:7]

    return R_orig, ph_orig, a_out, h_out_orig

In [335]:
# unpack_output(model(X))

In [336]:
def compute_economy(X, output):
    """
    Solves today's economy given state and output of neural network

    Parameters
    ----------
    output : _type_
        _description_
    """

    # Unpack inputs and outputs
    w = X[:, 2:5] # Distribution of tangible wealth
    R_orig, ph_orig, a_out, h_out_orig = unpack_output(output)

    # The network can predict negative values before it learns not to. 
    R = torch.maximum(R_orig, torch.ones_like(R_orig)*eps)
    ph = torch.maximum(ph_orig, torch.ones_like(ph_orig)*eps)
    h_out = torch.maximum(h_out_orig, torch.ones_like(h_out_orig)*lb)

    # Compute savings for oldest cohort
    a_out_J = -(Pih * ph * torch.unsqueeze(h_out[:, J-1], 1)) / R
    a_out_all = torch.cat([a_out, a_out_J], dim=1)

    # Compute consumption and impose non-negative constraint
    c_orig = w - ph*(1+delta)*h_out - a_out_all
    c = torch.maximum(c_orig, torch.ones_like(c_orig)*lb)

    # Compute aggregate savings and housing
    A_out = torch.sum(mu * a_out_all, dim=1, keepdims=True)
    H_out = torch.sum(mu * h_out, dim=1, keepdims=True)

    return R_orig, R, ph_orig, ph, a_out, a_out_all, h_out_orig, h_out, c_orig, c, A_out, H_out

In [337]:
# compute_economy(X, model(X))

In [338]:
def compute_next_state(X, output):
    """
    Computes the state for the next period, given the outputs of the neural network today.

    Parameters
    ----------
    outputs : tensors
        Outputs of neural network
    """

    _, R, _, ph, a_out, _, _, h_out, _, _, _, _ = compute_economy(X, output)

    m = torch.Tensor.size(X)[0]

    a_prime = torch.cat([torch.zeros([m, 1]), a_out], dim=1)
    h_prime = torch.cat([torch.zeros([m, 1]), h_out[:, 0:J-1]], dim=1)

    # Distribution of tomorrow's financial wealth
    w_prime = y + (R*a_prime + Pih*ph*h_prime) / (zeta_0*Gchi*Pic)

    # Tomorrow's extended state: catenate the parts together
    X_prime = torch.cat([
        R,
        ph,
        w_prime,
        a_prime,
        h_prime
        ], dim=1)        

    return X_prime

In [339]:
# compute_next_state(X, model(X))

In [340]:
def run_period(X):
    """
    Evaluate neural network given today's state and generate tomorrow's state.

    Parameters
    ----------
    X : tensor
        Today's extended state

    Returns
    -------
    tensor
        Tomorrow's extended state
    """

    # Evaluate neural network
    output = model(X)

    # Compute next state
    X_prime = compute_next_state(X, output)
    
    return X_prime

In [341]:
# run_period(X)

In [342]:
def euler_errors(X, output, X_prime, output_prime):
    """
    Compute error functions

    Parameters
    ----------
    X : _type_
        _description_
    output : _type_
        _description_
    X_prime : _type_
        _description_
    output_prime : _type_
        _description_

    Returns
    -------
    _type_
        _description_
    """
    # Compute today's economy
    R_orig, R, ph_orig, ph, _, _, h_out_orig, h_out, c_orig, c, A_out, H_out = compute_economy(X, output)

    # Compute tomorrow's economy
    R_prime_orig, R_prime, ph_prime_orig, ph_prime, _, _, h_prime_out_orig, h_prime_out, c_prime_orig, c_prime, A_prime_out, H_prime_out = compute_economy(X_prime, output_prime)

    # Euler equation 1 (Intertemporal)
    opt_euler1 = -1 + (c_prime[:, 1:J] / c[:, 0:J-1])*(R*betahat/(Gchi*Pic))**(-1/gamma)
    # opt_euler1 = grad_Uc(c[:, 0:J-1], h_out[:, 0:J-1]) - grad_Uc(c_prime[:, 1:J], h_prime_out[:, 1:J]) * R*betahat/(Gchi*Pic)

    # Euler equation 2 (Intratemporal)
    # opt_euler2 = -1 + (c/h_out)*(((1-alpha)/alpha)*ph*(1+delta-Pih/R))**(-1)
    opt_euler_intra = -1 + (c/h_out)*(((1-alpha)/alpha)*ph*(1+delta-Pih/R))**(-1)
    opt_euler_intra_prime = -1 + (c_prime/h_prime_out)*(((1-alpha)/alpha)*ph_prime*(1+delta-Pih/R_prime))**(-1)
    opt_euler2 = torch.cat([opt_euler_intra, opt_euler_intra_prime], dim=1)

    # Punishment for negative consumption
    orig_cons = torch.cat([c_orig, c_prime_orig], dim=1)
    opt_punish_cons = (1./eps) * torch.maximum(-1 * orig_cons, torch.zeros_like(orig_cons))

    # Punishment for negative housing
    orig_housing = torch.cat([h_out_orig, h_prime_out_orig], dim=1)
    opt_punish_housing = (1./eps) * torch.maximum(-1 * orig_housing, torch.zeros_like(orig_housing))

    # Punishment for negative interest rate
    orig_R = torch.cat([R_orig, R_prime_orig], dim=1)
    opt_punish_R = (1./eps) * torch.maximum(-1 * orig_R, torch.zeros_like(orig_R))

    # Punishment for negative house price
    orig_ph = torch.cat([ph_orig, ph_prime_orig], dim=1)
    opt_punish_ph = (1./eps) * torch.maximum(-1 * orig_ph, torch.zeros_like(orig_ph))

    # Market clearing condition for aggregate savings
    opt_market_a = A_out
    opt_market_a_prime = A_prime_out
    opt_market_A = torch.cat([opt_market_a, opt_market_a_prime], dim=1)
    # opt_market_A = A_out

    # Market clearing condition for aggregate housing
    opt_market_h = H_out - Hs
    opt_market_h_prime = H_prime_out - Hs
    opt_market_H = torch.cat([opt_market_h, opt_market_h_prime], dim=1)
    # opt_market_H = H_out - Hs

    # Concatenate the equilibrium functions
    combined_opt = [opt_euler1, opt_euler2, opt_punish_cons, opt_punish_housing, opt_punish_R, opt_punish_ph, opt_market_A, opt_market_H]
    opt_predict = torch.cat(combined_opt, dim=1)

    # Define the "correct" outputs. For all equilibrium functions, the correct outputs is zero.
    opt_correct = torch.zeros_like(opt_predict)

    return opt_predict, opt_correct

In [343]:
def simulate(X):
    """
    Simulate and return euler errors

    Parameters
    ----------
    X : _type_
        _description_

    Returns
    -------
    _type_
        Euler errors
    """

    # Evaluate neural network given today's state
    output = model(X)

    # Compute state tomorrow
    X_prime = compute_next_state(X, output)

    # Evaluate neural network given tomorrow's state
    output_prime = model(X_prime.float())

    # Compute euler errors and other equilibrium conditions
    opt_predict, opt_correct = euler_errors(X, output, X_prime, output_prime)

    return opt_predict, opt_correct

In [344]:
# opt_predict, opt_correct = simulate(X)

# 4. Training

In [345]:
# Generate a random economically feasible starting point
x_start = torch.rand(size=(1, input_size))

num_episodes = 100
len_episodes = 16
epochs_per_episode = 4
minibatch_size = 8
num_minibatches = int(len_episodes / minibatch_size)
lr = 1e-2

optimizer = torch.optim.Adam(model.parameters(), lr=lr, amsgrad=True, capturable=True)

In [346]:
def simulate_episodes(x_start, episode_length):
    """Simulate an episode for a given starting point using the current
       neural network state.

    Args:
        x_start: Starting state to simulate forward from,
        episode_length: Number of steps to simulate forward,

    Returns:
        X_episodes: Tensor of states [z, k] to train on (training set).
    """
    
    dim_state = torch.Tensor.size(x_start)[1]

    # Initialise empty array of episodes
    X_episodes = torch.zeros([episode_length, dim_state])
    X_episodes[0, :] = x_start
    X_old = x_start

    for t in range(1, episode_length):
        X_new = run_period(X_old.float())
        
        # Append it to the dataset
        X_episodes[t, :] = X_new
        X_old = X_new

    return X_episodes

In [347]:
# simulate_episodes(x_start, 1000)

In [348]:
def train_step(X_train):

    X_train = torch.utils.data.DataLoader(X_train, batch_size=minibatch_size)
    model.train()

    for epoch in range(epochs_per_episode):

        for X_batch in X_train:

            # Forward pass of the model to get Euler errors
            opt_predict, opt_correct = simulate(X_batch)

            # Compute loss
            loss = torch.nn.functional.mse_loss(opt_predict, opt_correct)             

            # Use gradient tape to retrieve gradients of trainable variables with respect to loss
            optimizer.zero_grad()
            loss.backward()
            torch.autograd.set_detect_anomaly(True)
            optimizer.step()

            # print(f"loss: {torch.log(loss):>7f}")

    return loss

In [349]:
# train_step(x_start)

In [350]:
torch.set_printoptions(precision=4, sci_mode=False)
np.set_printoptions(precision=4, linewidth=100, suppress=True)

def training_algorithm(x_start):

    num_episodes = 1000 #try 30000
    loss_list = np.empty(num_episodes)

    # Training Loops
    for episode in range(num_episodes):

        # Simulate episodes
        X_episodes = simulate_episodes(x_start, len_episodes)
        X_episodes = torch.Tensor.detach(X_episodes)
        
        # Train model
        loss_value = train_step(X_episodes)

        # Update starting episode
        x_start = X_episodes[-1, :].reshape([1, -1])

        # Store losses, euler errors
        loss_list[episode] = torch.log(loss_value)
        
        # Log
        if episode % 1 == 0:
            print(f"Episode {episode}: log10(loss): {torch.log(loss_value):.5f}")
            print(torch.Tensor.cpu(x_start).numpy())
          
    return x_start, loss_list

In [351]:
x_final, loss_list = training_algorithm(x_start)

Episode 0: log10(loss): 1.53755
[[ 0.0012  0.1099  0.8222  1.2334  0.2034  0.     -0.079   0.1346  0.      0.001   0.0238]]
Episode 1: log10(loss): 2.42283
[[ 0.2883  0.3892  0.8222  1.3642  0.2969  0.     -0.1429  0.0104  0.      0.3675  0.1924]]
Episode 2: log10(loss): 2.74060
[[ 0.337   0.6083  0.8222  1.4802  0.4563  0.     -0.0243 -0.135   0.      0.4012  0.3582]]
Episode 3: log10(loss): 2.65693
[[ 0.3638  0.7126  0.8222  1.5244  0.5779  0.      0.0609 -0.2356  0.      0.3872  0.4606]]
Episode 4: log10(loss): 2.33198
[[ 0.3971  0.721   0.8222  1.5243  0.6108  0.      0.0974 -0.2939  0.      0.374   0.507 ]]
Episode 5: log10(loss): 1.95598
[[ 0.4331  0.6781  0.8222  1.5056  0.5835  0.      0.1011 -0.3228  0.      0.368   0.5202]]
Episode 6: log10(loss): 1.63024
[[ 0.464   0.6215  0.8222  1.4826  0.5377  0.      0.0907 -0.3346  0.      0.3663  0.519 ]]
Episode 7: log10(loss): 1.37430
[[ 0.4875  0.5692  0.8222  1.4612  0.4947  0.      0.0769 -0.3375  0.      0.3664  0.5134]]
Episode 